In [1]:
from math import sqrt, floor
def extended_euclidean(a,b):
    """Returns gcd(a,b) along with integers s,t such that gcd(a,b)=as+bt using
     the extended euclidean algorithm."""

    r_old, r = a,b
    s_old, s = 1,0
    t_old, t = 0,1
    while r > 0:
        q = r_old // r
        r_old, r = r, r_old%r
        s_old, s = s, s_old-s*q
        t_old, t = t, t_old-t*q
    return r_old, s_old, t_old

def square_and_multiply(g,x,m):
    """Returns g^x modulo m using the low-storage square-and-multiply algorithm.
    """
    b = 1
    a = g
    while x>0:
        if x%2 == 1:
            b = b*a % m
        a = (a**2) % m
        x = x // 2
    return b

def modular_inverse(a,m):
    """Returns an integer b such that a*b=1 mod m if gcd(a,m)=1"""
    g,s,t = extended_euclidean(a,m)
    if g != 1:
        raise ValueError('Input must be coprime')
    return s%m

def babysteps_giantsteps(g,h,p,N=None):
    """Returns the discrete log of h with respect to the base g modulo p, where g has order N modulo p.
    """
    if not N: N = p-1
    n = floor((sqrt(N)))+1
    # make a dictionary of the babysteps for fast lookup of the exponent
    babystep = 1
    babysteps_table = {}
    for i in range(n):
        babysteps_table[babystep] = i
        babystep = (babystep * g) % p
    g_inv = modular_inverse(g,p)
    g_n_inv = pow(g_inv, n, p)
    # giantsteps: h*g^(-jn) for j in [0..n-1]
    giantstep = h
    for j in range(n+1):
        i = babysteps_table.get(giantstep)
        #i==None if giantstep is not a babystep, otherwise i satisfies g^i == giantstep
        if not i == None:
            return i+n*j
        giantstep = (giantstep * g_n_inv) % p
    print("no solution!")
    return None

In [2]:
def CRT(moduli, values):
    """Returns an intenger n such that n modulo moduli[i] = values[i].
    Input: moduli - a list of k coprime positive integers
           values - a list of k integers
    """
    #YOUR CODE GOES HERE
    
    c = values[0];
    m = moduli[0];
    k = len(moduli);
    
    for i in range(1, k):
        for j in range(1, 200):
            y = modular_inverse(m, moduli[i]) * (values[i] - c) % moduli[i];
            if (isinstance(y, int)):
                break;
        c = c + m * y;
        m = m * moduli[i];
        
    return (c%m);

In [3]:
CRT([3,7,16], [2,3,4])

164

In [3]:
def q_power_dlog(g,h,p,q,e):
    """Returns the discrete logarithm of g modulo p of h, where g has
    order q^e modulo p."""
    #YOUR CODE GOES HERE
    
    newq = q ** (e - 1);
    
    a = square_and_multiply(g,newq,p);
    b0 = square_and_multiply(h,newq,p);
    x0 = babysteps_giantsteps(a,b0,p,q);
    
    xl = x0;
    
    for i in range(1, e):
        newi = q ** (e - i - 1);
        ## tempb = h * (g ** (-xl * q ** i));
        ## bi = square_and_multiply(tempb, newi, p);
        bi = pow((pow(g, -1 * xl, p) * h), newi, p);
        tempx = babysteps_giantsteps(a,bi,p,q);
        xl = xl + tempx * (q ** i);
    
    return xl;

In [15]:
newq = 5 ** (45 - 1);
h = pow(36,12345678910111213141516171819201,56843418860808014869689941406251);
a = square_and_multiply(36,newq,56843418860808014869689941406251);
b0 = square_and_multiply(h,newq,56843418860808014869689941406251);
print(newq);
print(h);
print(a);
print(b0);

5684341886080801486968994140625
17994056186373649813734623444563
30765770345747974325897255273242
30765770345747974325897255273242


In [16]:
babysteps_giantsteps(a,b0,56843418860808014869689941406251,5);

In [17]:
p = 2 * (3 ** 9) + 1;
g = 9;
h = 9 ** 5;
q = 3;
e = 9;
q_power_dlog(g,h,p,q,e)

5

In [18]:
p = 56843418860808014869689941406251 # p = 2*5^(45) + 1 is prime
g = 36  # 6 is primitive, 36 has order 5^45 
q = 5
e = 45
x = 12345678910111213141516171819201
h = pow(g,x,p)

q_power_dlog(g,h,p,q,e)

12345678910111213141516171819201

In [24]:
def pohlig_hellman(g,h,p,prime_divisors):
    """Returns the discrete logarithm of h with respect to the base g modulo p,
    where p = q_1^e_1 * ... * q_t^e^_t, and prime_divisors = [[q_1,e_1],...,[q_t,e_t]].
    """
    #YOUR CODE GOES HERE
    
    xi = [None] * len(prime_divisors);         ## xi is qi in HW6 file
    for j in range(0, len(prime_divisors)):
        xi[j] = 0;
        
    yi = [None] * len(prime_divisors);         ## yi is ei in HW6 file
    for j in range(0, len(prime_divisors)):
        yi[j] = 0;
    
    for i in range (0, len(prime_divisors)):
        ai = prime_divisors[i];
        xi[i] = ai[0];
        yi[i] = ai[1];
    ## End of elements extraction
    
    
    Ni = [None] * len(xi);
    for j in range(0, len(xi)):
        Ni[j] = 0;
        
    for i in range(0, len(xi)):
        Ni[i] = xi[i] ** yi[i];
    
    N = 1;
    for i in range(0, len(xi)):
        N = N * Ni[i]
    
    t = len(xi);
    zi = [None] * t;             ## zi is yi in HW6 file
    for j in range(0, t):
        zi[j] = 0;
    for i in range (0, t):
        gi = square_and_multiply(g, N//Ni[i], p);
        hi = square_and_multiply(h, N//Ni[i], p);
        zi[i] = q_power_dlog(gi,hi,p,xi[i],yi[i]);
    
    c = CRT(Ni, zi);
    
    return (c%N);

In [25]:
prime_divisors = [[2,1],[3,19],[5,19]];
p = 44336756401062011718751;
g = 3;
x = 12345678910111213141516;
h = pow(g,x,p);

pohlig_hellman(g,h,p,prime_divisors)

12345678910111213141516

In [18]:
p = 211
g = 2
h = 2 ** 5
prime_divisors = [[2, 1], [3, 1], [5, 1], [7, 1]]

pohlig_hellman(g,h,p,prime_divisors)

[2, 3, 5, 7]
210
zi [1, 2, 0, 5]
Ni [2, 3, 5, 7]
5


5

In [9]:
CRT([2, 3, 5, 7], [1, 1, 2, 2])

37

In [14]:
CRT([2, 1162261467, 19073486328125], [1, 581130733, 9536743164062])

11084189100265502929687

In [18]:
a = [[2, 1], [3, 1], [5, 1], [7, 1]];

xi = [None] * len(a);
for j in range(0, len(a)):
    xi[j] = 0;
    
yi = [None] * len(a);
for k in range(0, len(a)):
    yi[k] = 0;

for i in range (0, len(a)):
    ai = a[i];
    xi[i] = ai[0];
    yi[i] = ai[1];

print(xi)
print(yi)

[2, 3, 5, 7]
[1, 1, 1, 1]


In [25]:
pow(3,-1,7)

5

In [12]:
44336756401062011718751 - 3082519042187505455

44333673882019824213296

In [14]:
3 ** 19

1162261467

In [19]:
2 * 1162261467 * 19073486328125

44336756401062011718750